In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import joblib

from scipy.stats import chi2_contingency
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE, SelectKBest, chi2
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc

In [2]:
test_data = pd.read_csv('../data/test_transformed.csv').drop(['Name', 'PassengerId'], axis=1)

submission_sample = pd.read_csv('../data/sample_submission.csv')

model = joblib.load('xgbclassifier_model_spaceship_titanic.pkl')

In [12]:
model

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('encoder',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder())]),
                                                                         Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'PassengerNumber',
       'GroupSize', 'CabinDeck', 'CabinSide', 'GroupNumberBuckets',
       'CabinNumberBuckets'],...
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=None, ...))]),
             param_grid={'classifier__learning_rate': [0.1, 0.2, 0.3],
                         'classifier__max_depth': [4, 6, 8],
                         'classifier__n_estimators': [50, 100],
                         'classifier__subsample': [0.5, 0.7, 0.9]})

In [5]:
test_data = test_data.drop(['Name', 'PassengerId'], axis=1)
test_data['VIP'] = test_data['VIP'].replace({True: 'True', False: 'False'})
test_data['CryoSleep'] = test_data['CryoSleep'].replace({True: 'True', False: 'False'})

In [7]:
predictions = model.predict(test_data)
predictions

array([1, 0, 1, ..., 1, 1, 1])

In [10]:
final_submission = submission_sample.copy()
final_submission['Transported'] = predictions
final_submission['Transported'] = final_submission['Transported'].replace({1: True, 0: False})
final_submission.to_csv('../submissions/final_submission.csv', index=None)
final_submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True
